In [26]:
import numpy as np
# import cupy as np
from __future__ import annotations

In [27]:
class Tensor:

  def __init__(self, list: list | np.ndarray) -> None:
    self.name: str = ""
    self.values: np.ndarray = np.array(list, dtype="float32")
    self.parents: tuple[Tensor, Tensor] = (None, None)
    self.gradient: np.array = np.zeros_like(self.values)
    self._backward: function = lambda *args: None
    self.visited = False

  def __add__(self, other: Tensor) -> Tensor:
    child = Tensor(self.values + other.values)
    child.parents = (self, other)
    def _backward() -> None:
      self.gradient += np.ones_like(self.values) * child.gradient
      other.gradient += np.ones_like(other.values) * child.gradient
    child._backward = _backward
    return child
  
  def __sub__(self, other: Tensor) -> Tensor:
    return self.__add__(-other)

  def __mul__(self, other: Tensor) -> Tensor:
    child = Tensor(self.values * other.values)
    child.parents = (self, other)
    def _backward() -> None:
      self.gradient += other.values * child.gradient # mistake made using Tensor object instead of tensor value
      other.gradient += self.values * child.gradient # keep in mind whether dealing with object object.values or object.gradient
    child._backward = _backward
    return child

  def __truediv__(self, other: Tensor) -> Tensor:
    return self.__mul__(other ** -1)

  def __neg__(self) -> Tensor:
    child = Tensor(-self.values)
    child.parents = (self, None)
    def _backward():
      self.gradient += -np.ones_like(self.values) * child.gradient
    child._backward = _backward
    return child

  def __pow__(self, other: float) -> Tensor:
    child = Tensor(self.values ** other)
    child.parents = (self, None)
    def _backward():
      self.gradient += other * self.values ** (other - 1) * child.gradient
    child._backward = _backward
    return child
  
  def __matmul__(self, other: Tensor) -> Tensor:
    child = Tensor(self.values @ other.values)
    child.parents = (self, other)
    def _backward():
      # self: A x B - other B x C - child A x C
      self.gradient = child.gradient @ other.values.T # A x C @ C x B => A x B
      other.gradient = self.values.T @ child.gradient # B x A @ A x C => B x C
    child._backward = _backward
    return child

  def exp(self) -> Tensor:
    child = Tensor(np.exp(self.values))
    child.parents = (self, None)
    def _backward():
      self.gradient = child.values * child.gradient
    child._backward = _backward
    return child

  def tanh(self) -> Tensor:
    child = Tensor(np.tanh(self.values))
    child.parents = (self, None)
    def _backward():
      self.gradient += (1 - child.values ** 2) * child.gradient
    child._backward = _backward
    return child

  def sum(self) -> Tensor:
    child = Tensor([self.values.sum()])
    child.parents = (self, None)
    def _backward() -> None:
      self.gradient += np.ones_like(self.values) * child.gradient
    child._backward = _backward
    return child

  def topological_sort(self) -> list[Tensor]:
    dfs_sort: list[Tensor] = []
    def dfs(node: Tensor):
      if node.parents[0]:
        dfs(node.parents[0])
      if node.parents[1]:
        dfs(node.parents[1])
      if not node.visited:
        dfs_sort.append(node); node.visited = True
    dfs(self); return list(reversed(dfs_sort))

  def backward(self):
    self.gradient = np.ones_like(self.values) # gradient w.r.t self
    for node in self.topological_sort():
      node._backward()
      # print(node)

  def zero_grad(self):
    self.gradient = np.zeros_like(self.values)
    self.visited = False
  
  def __repr__(self) -> str:
    return f"{self.name}: {self.values} : {self.gradient}"

In [28]:
np.random.seed(42)
a = Tensor(np.random.uniform(size=(500, 1000)))
b = Tensor(np.random.uniform(size=(1000, 20)))
c = Tensor(np.random.uniform(size=(500, 20)))

d = a @ b
e = d / c ** 2
l = e.sum()
print(l)
l.backward()
print(a.gradient)


: [1.5470199e+10] : [0.]
[[1.3207467e+03 3.4329600e+03 3.5435869e+03 ... 2.8773391e+03
  5.5576836e+03 3.8428843e+03]
 [3.8589172e+02 9.3177711e+01 1.3299084e+02 ... 4.2933572e+02
  2.6310379e+02 3.9735739e+02]
 [7.1169453e+04 7.6358018e+03 8.9608062e+04 ... 7.1529531e+04
  6.4949184e+04 6.8439097e+03]
 ...
 [5.1556559e+04 3.0674984e+05 5.8992612e+05 ... 1.6457391e+05
  6.2774131e+05 7.4499312e+05]
 [3.4821860e+02 2.7197885e+02 2.9004333e+02 ... 2.2436359e+02
  2.2142862e+02 2.4646594e+02]
 [8.1866264e+01 6.8387070e+01 9.1307175e+01 ... 8.9380768e+01
  9.2049667e+01 9.0049286e+01]]


In [29]:
import torch
np.random.seed(42)
a = torch.tensor(np.random.uniform(size=(500, 1000)), requires_grad=True)
b = torch.tensor(np.random.uniform(size=(1000, 20)), requires_grad=True)
c = torch.tensor(np.random.uniform(size=(500, 20)), requires_grad=True)

d = a @ b; d.retain_grad()
e = d / c ** 2; e.retain_grad()
l = e.sum(); l.retain_grad()
l.backward()
# print(l)
# tensors = list(reversed([a, b, c, d, e, l]))
# for tensor in tensors:
#   print(tensor)
print(a.grad)

tensor([[1.3207e+03, 3.4330e+03, 3.5436e+03,  ..., 2.8773e+03, 5.5577e+03,
         3.8429e+03],
        [3.8589e+02, 9.3178e+01, 1.3299e+02,  ..., 4.2934e+02, 2.6310e+02,
         3.9736e+02],
        [7.1169e+04, 7.6358e+03, 8.9608e+04,  ..., 7.1530e+04, 6.4949e+04,
         6.8439e+03],
        ...,
        [5.1557e+04, 3.0675e+05, 5.8993e+05,  ..., 1.6457e+05, 6.2774e+05,
         7.4499e+05],
        [3.4822e+02, 2.7198e+02, 2.9004e+02,  ..., 2.2436e+02, 2.2143e+02,
         2.4647e+02],
        [8.1866e+01, 6.8387e+01, 9.1307e+01,  ..., 8.9381e+01, 9.2050e+01,
         9.0049e+01]], dtype=torch.float64)


In [204]:
a = Tensor([1, 2, 3]); a.name = "a"
b = Tensor([2, 4, 6]); b.name = "b"
c = Tensor([3, 6, 9]); c.name = "c"
d = a * b; d.name = "d"
dn = -d; dn.name = "dn"
e = dn.exp() - c; e.name = "e"
e2 = e ** 2; e2.name = "e2"
f = e2 / a.tanh(); f.name = "f"
l = f.sum(); l.name = "l"
l.backward()

l: [129.51688] : [1.]
f: [10.775167 37.339153 81.40256 ] : [1. 1. 1.]
: [1.3130352 1.0373148 1.0049698] : [ 8.206305 35.99597  81.      ]
: [0.7615942 0.9640276 0.9950548] : [-14.148174 -38.73245  -81.80711 ]
e2: [ 8.206305 35.99597  81.      ] : [1.3130352 1.0373148 1.0049698]
e: [-2.8646648 -5.9996643 -9.       ] : [ -7.522812 -12.447081 -18.089457]
: [-3. -6. -9.] : [ -7.522812 -12.447081 -18.089457]
c: [3. 6. 9.] : [ 7.522812 12.447081 18.089457]
: [1.3533528e-01 3.3546262e-04 1.5229981e-08] : [ -7.522812 -12.447081 -18.089457]
dn: [ -2.  -8. -18.] : [-1.0181018e+00 -4.1755303e-03 -2.7550209e-07]
d: [ 2.  8. 18.] : [1.0181018e+00 4.1755303e-03 2.7550209e-07]
b: [2. 4. 6.] : [1.0181018e+00 8.3510606e-03 8.2650627e-07]
a: [1. 2. 3.] : [-3.9056666  -2.7197769  -0.80710727]


In [205]:
# comparing with pytorch
import torch
a = torch.tensor([1, 2, 3], dtype=float, requires_grad=True)
b = torch.tensor([2, 4, 6], dtype=float, requires_grad=True)
c = torch.tensor([3, 6, 9], dtype=float, requires_grad=True)
d = a * b; d.retain_grad()
dn = -d; dn.retain_grad()
e = dn.exp() - c; e.retain_grad()
e2 = e ** 2; e2.retain_grad()
f = e2 / a.tanh(); f.retain_grad()
l = f.sum(); l.retain_grad()
l.backward()

In [206]:
tensors = list(reversed([a, b, c, d, e, e2, f, l]))
for tensor in tensors:
  print(tensor.grad)

tensor(1., dtype=torch.float64)
tensor([1., 1., 1.], dtype=torch.float64)
tensor([1.3130, 1.0373, 1.0050], dtype=torch.float64)
tensor([ -7.5228, -12.4471, -18.0895], dtype=torch.float64)
tensor([1.0181e+00, 4.1755e-03, 2.7550e-07], dtype=torch.float64)
tensor([ 7.5228, 12.4471, 18.0895], dtype=torch.float64)
tensor([1.0181e+00, 8.3511e-03, 8.2651e-07], dtype=torch.float64)
tensor([-3.9057, -2.7198, -0.8071], dtype=torch.float64)


In [196]:
tensors = list(reversed([a, b, c, d, e, e2, f, l]))
for tensor in tensors:
  print(tensor)

tensor(968.7626, dtype=torch.float64, grad_fn=<SumBackward0>)
tensor([ 32.8259, 203.3137, 732.6230], dtype=torch.float64,
       grad_fn=<DivBackward0>)
tensor([ 25., 196., 729.], dtype=torch.float64, grad_fn=<PowBackward0>)
tensor([ -5., -14., -27.], dtype=torch.float64, grad_fn=<SubBackward0>)
tensor([ 2.,  8., 18.], dtype=torch.float64, grad_fn=<MulBackward0>)
tensor([3., 6., 9.], dtype=torch.float64, requires_grad=True)
tensor([2., 4., 6.], dtype=torch.float64, requires_grad=True)
tensor([1., 2., 3.], dtype=torch.float64, requires_grad=True)
